<b>EXERCICE 1</b>
***

In [1]:
import sys
import pandas as pd
import numpy as np

<B>Chargement et création du dataframe avec les données de différentes variables (population, education, dépenses ...)  dans chaque pays, sur différentes années</b>

In [2]:
path_EdStatsData = "C:/Users/victo/Desktop/Formation Data Scientist OpenClassRoom/Cours2/EdStatsData.csv"
df_EdStatsData = pd.read_csv(path_EdStatsData)

path_EdStatsSeries = "C:/Users/victo/Desktop/Formation Data Scientist OpenClassRoom/Cours2/EdStatsSeries.csv"
df_EdStatsSeries = pd.read_csv(path_EdStatsSeries)

path_EdStatsCountry = "C:/Users/victo/Desktop/Formation Data Scientist OpenClassRoom/Cours2/EdStatsCountry.csv"
df_EdStatsCountry = pd.read_csv(path_EdStatsCountry)

path_EdStatsCountrSeries = "C:/Users/victo/Desktop/Formation Data Scientist OpenClassRoom/Cours2/EdStatsCountry-Series.csv"
df_EdStatsCountrySeries = pd.read_csv(path_EdStatsCountrSeries)

# On conserve que les pays (avec monnaie et Region), les autres étant des régions sans monnaie unique
list_real_country = df_EdStatsCountry[
    (df_EdStatsCountry["Currency Unit"].notna()) & (df_EdStatsCountry["Region"].notna())
]["Short Name"]
list_real_country.name = "Country Name"

# On ne conserve que les "vrais" pays, cad qui ne sont pays des zones géographiques
df_EdStatsData = df_EdStatsData[df_EdStatsData["Country Name"].isin(list_real_country)]

# nbres lignes et colonnes
df_EdStatsData.shape

# Les différents métiers concernés par les indicateurs
df_EdStatsSeries["Topic"].value_counts()
metiers = df_EdStatsSeries["Topic"].unique()

<b> Remise en forme du dataframe </b>

In [3]:
annees = [col for col in df_EdStatsData.columns if col.isnumeric()]

# Compter les valeurs non-NaN pour chaque année
non_nan_counts = df_EdStatsData[annees].count()
# Transformer en DataFrame clair
result = non_nan_counts.reset_index()
result.columns = ["année", "nb_non_nan"]

# Créer la liste d'années de 2000 à 2015, qui sont les années avec le plus de donnéées renseignées
annees = np.arange(2010, 2015).tolist()
# Convertir chaque année en string
annees_str = list(map(str, annees))
# Sélectionner uniquement les colonnes années dans cette plage
annees_filtrees = [
    col
    for col in df_EdStatsData.columns
    if col.isnumeric() and 2010 <= int(col) <= 2015
]
# Colonnes non-années
autres_colonnes = [col for col in df_EdStatsData.columns if not col.isnumeric()]
# Combiner les deux listes
colonnes_a_garder = autres_colonnes + annees_filtrees
# Créer le DataFrame filtré
df_EdStatsData = df_EdStatsData[colonnes_a_garder]

# backfill en reprenant l'année précedente
df_EdStatsData[annees_filtrees] = df_EdStatsData[annees_filtrees].bfill(axis=1)

# === 0. Passage en format long ===
df_EdStatsData = df_EdStatsData.melt(
    id_vars=["Country Name", "Indicator Name"],
    value_vars=annees_filtrees,
    var_name="Année",
    value_name="Valeur",
)

Pivot de la table

In [4]:
# Pivot de la table pour avoir en ligne les pays et en colonne les indicateurs
df_EdStatsData = pd.pivot_table(
    df_EdStatsData,
    values="Valeur",
    index="Country Name",  # lignes = pays
    columns="Indicator Name",  # colonnes = indicateurs
    aggfunc="mean",  # moyenne sur toutes les années
)

On filtre les colonnes avec trop de Nan

In [5]:
# on calcule le pourcentage de NaN par colonne
na_ratio = df_EdStatsData.isna().mean()

# on garde uniquement les colonnes dont le % NaN <= 20 %
df_EdStatsData = df_EdStatsData.loc[:, na_ratio <= 0.20]

# Liste des colonnes à garder, en fonction de notre problématique
colonnes_utiles = [
    "Attainment",
    "Teachers",
    "Literacy",
    "Primary",
    "Secondary",
    "Tertiary",
    "Education Equality",
    "Infrastructure: Communications",
    "Population",
    # ajouter toutes les colonnes pertinentes pour ton analyse
]

# Filtrer les indicateurs du DataFrame
df_EdStatsSeries = df_EdStatsSeries[df_EdStatsSeries["Topic"].isin(colonnes_utiles)]
list_indicators = df_EdStatsSeries["Indicator Name"].unique().tolist()
existing_cols = [c for c in list_indicators if c in df_EdStatsData.columns]
df_EdStatsData = df_EdStatsData[existing_cols]

In [6]:
# Liste de mots-clés par axe
keywords = ["demography", "education", "internet", "economy"]

# Colonnes correspondant aux mots-clés
cols_match = [
    c for c in df_EdStatsData.columns if any(k.lower() in c.lower() for k in keywords)
]

# Limite à 10 indicateurs max
cols_match = cols_match[:10]

df_filtered = df_EdStatsData[cols_match]
df_filtered.columns

Index(['Internet users (per 100 people)',
       'Official entrance age to primary education (years)',
       'Theoretical duration of primary education (years)',
       'Enrolment in primary education, both sexes (number)',
       'Enrolment in primary education, female (number)',
       'Percentage of students in primary education who are female (%)',
       'Pupil-teacher ratio in primary education (headcount basis)',
       'Gross intake ratio to Grade 1 of primary education, female (%)',
       'Gross intake ratio to Grade 1 of primary education, male (%)',
       'Gross intake ratio to Grade 1 of primary education, both sexes (%)'],
      dtype='object', name='Indicator Name')

Score Pondéré

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = df_filtered.copy()
df_scaled[cols_match] = scaler.fit_transform(df_filtered[cols_match])

# Exemple : pondération égale ou définie par axe
weights = {col: 1 / len(cols_match) for col in cols_match}

df_scaled["Score"] = sum(df_scaled[col] * w for col, w in weights.items())

Visualisation sur Carte Mondiale

In [8]:
import plotly.express as px
import plotly.io as pio

import nbformat

pio.renderers.default = "browser"
fig = px.choropleth(
    df_scaled.reset_index(),
    locations="Country Name",
    locationmode="country names",
    color="Score",
    color_continuous_scale="Viridis",
    title="Carte mondiale des scores pondérés",
)

fig.show()

C:\Users\victo\AppData\Local\Temp\ipykernel_162176\1507359622.py:7: DeprecationWarning:

The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.

